<a href="https://colab.research.google.com/github/sanjayKumar-dev/languageTranslator/blob/main/newlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
batch_size = 64
latent_dim = 256
num_samples = 20000

In [ ]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open('fra.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 74
Number of unique output tokens: 102
Max sequence length for inputs: 19
Max sequence length for outputs: 59


In [ ]:
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_in_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_in_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 74)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 102)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        338944      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_in_data shape: (20000, 19, 74)
decoder_in_data shape: (20000, 59, 102)
decoder_target_data shape: (20000, 59, 102)


In [ ]:
model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')

model.fit([encoder_in_data, decoder_in_data], decoder_target_data, batch_size = batch_size, epochs=51, validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/51
250/250 [==============================] - 90s 350ms/step - loss: 0.7294 - val_loss: 0.7581
Epoch 2/51
250/250 [==============================] - 87s 349ms/step - loss: 0.5957 - val_loss: 0.7112
Epoch 3/51
250/250 [==============================] - 87s 347ms/step - loss: 0.5662 - val_loss: 0.6962
Epoch 4/51
250/250 [==============================] - 88s 351ms/step - loss: 0.5487 - val_loss: 0.6700
Epoch 5/51
250/250 [==============================] - 88s 352ms/step - loss: 0.5289 - val_loss: 0.6537
Epoch 6/51
250/250 [==============================] - 87s 350ms/step - loss: 0.5141 - val_loss: 0.6416
Epoch 7/51
250/250 [==============================] - 87s 349ms/step - loss: 0.6763 - val_loss: 0.8033
Epoch 8/51
250/250 [==============================] - 87s 349ms/step - loss: 0.6093 - val_loss: 0.7242
Epoch 9/51
250/250 [==============================] - 87s 350ms/step - loss: 0.6531 - val_loss: 0.7534
Epoch 10/51
250/250 [==============================] - 88s 352ms/step - l

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())


def decode_sequence(input_seq):
   
    states_value = encoder_model.predict(input_seq)

    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    target_seq[0, 0, target_token_id['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va der coureux.

-
Input sentence: Run!
Decoded sentence: Courez ?

-
Input sentence: Run!
Decoded sentence: Courez ?

-
Input sentence: Wow!
Decoded sentence: Varaide.

-
Input sentence: Fire!
Decoded sentence: Au feur.

-
Input sentence: Help!
Decoded sentence: Aiendez !

-
Input sentence: Jump.
Decoded sentence: Vour ?

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

